In [1]:
from IPython.display import HTML, display

import json
from bson.json_util import dumps
import pandas as pd
from fastparquet import ParquetFile,write
import time
import datetime
import copy
import numpy as np
import math

import pandas as pd
import re

In [2]:
pf = ParquetFile('do512_10-3-2019-10-3-2018.parquet')
df = pf.to_pandas(index=False)
df.head()

,id,permalink,presented_by,title,description,excerpt,category,category_param,conference_category_param,begin_time,...,rsvp.require_terms_agreement,rsvp.terms_label,rsvp.terms_text,rsvp.action,imagery.aws.image,imagery.aws.custom_share_image_w_630_h_1200,dow,repeat_until,repeating,time_tba
0,11049447,/events/2019/10/3/gary-clark-jr-tickets,Official 2019 ACL Fest Late Night Show:,Gary Clark Jr,<p><br></p><p>Stubb's Outdoor&nbsp;</p><p><br>...,Stubb's Outdoor \n\nTo sum up Gary Clark Jr. i...,Music,music,,2019-10-03T19:00:00-05:00,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
1,11049456,/events/2019/10/3/hippie-sabotage-w-sebastian-...,Official 2019 ACL Fest Late Night Show:,Hippie Sabotage w/ Sebastian Paul,"<p><br></p><p></p><p><a target=""_blank"" href=""...",Hippie Sabotage w/ SEBASTIAN PAUL 2019 ACL Fes...,Music,music,,2019-10-03T20:00:00-05:00,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
2,11049430,/events/2019/10/3/fidlar-w-xetas-tickets,Official 2019 ACL Fest Late Night Show:,FIDLAR w/ XETAS,<p><br></p><p>FIDLAR are slackers at heart. Th...,FIDLAR are slackers at heart. The only thing t...,Music,music,,2019-10-03T18:30:00-05:00,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
3,11186444,/events/2019/10/3/that-jokes-new-comedy-open-m...,Misfits Comedy Presents,That Joke's New! Comedy Open-Mic,<p><br></p><p></p><p>Got jokes? Come make us l...,Got jokes? Come make us laugh at this weekly T...,Comedy,comedy,,2019-10-03T20:00:00-05:00,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
4,11141496,/events/2019/10/3/hear-me-bitch-90s-birthday-b...,,Hear Me Bitch 90’s Birthday Bash Comedy Show,<div>Hear Me Bitch 90’s Birthday Bash Comedy S...,Hear Me Bitch 90’s Birthday Bash Comedy Show \...,Comedy,comedy,,2019-10-03T19:30:00-05:00,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN


In [40]:
# drop some insufficient columns
#df.columns
newcols=['id', 'presented_by', 'title',
       'category','begin_time','past',
       'end_time', 'votes', 'ticket_info','is_eventbrite', 'amazon_affiliate', 'eventbrite_id',
       'is_free','rating', 'doors', 'popularity', 'sold_out', 'favorite', 'artists',
       'rsvp','eager',
       'venue.id', 'venue.title','venue.latitude',
       'venue.longitude', 'venue.popularity', 'venue.address', 'venue.city',
       'venue.state', 'venue.zip','giveaway.action', 'giveaway.ask21']

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41865 entries, 0 to 41864
Columns: 111 entries, id to time_tba
dtypes: bool(17), float16(23), float64(7), int64(4), object(60)
memory usage: 25.2+ MB


In [41]:
ndf=df[newcols]

In [42]:
ndf.head()

,id,presented_by,title,category,begin_time,past,end_time,votes,ticket_info,is_eventbrite,...,venue.title,venue.latitude,venue.longitude,venue.popularity,venue.address,venue.city,venue.state,venue.zip,giveaway.action,giveaway.ask21
0,11049447,Official 2019 ACL Fest Late Night Show:,Gary Clark Jr,Music,2019-10-03T19:00:00-05:00,False,None,784,All Ages,False,...,Stubb's,30.268458,-97.736175,15.0,801 Red River,Austin,TX,78701,None,NaN
1,11049456,Official 2019 ACL Fest Late Night Show:,Hippie Sabotage w/ Sebastian Paul,Music,2019-10-03T20:00:00-05:00,False,None,223,18+,False,...,Emo's,30.240266,-97.728516,15.0,2015 E. Riverside Dr.,Austin,TX,78741,None,NaN
2,11049430,Official 2019 ACL Fest Late Night Show:,FIDLAR w/ XETAS,Music,2019-10-03T18:30:00-05:00,False,None,163,All Ages,False,...,Historic Scoot Inn,30.262141,-97.729385,15.0,1308 E. Fourth,Austin,TX,78702,None,NaN
3,11186444,Misfits Comedy Presents,That Joke's New! Comedy Open-Mic,Comedy,2019-10-03T20:00:00-05:00,False,None,151,Free,False,...,Halcyon (Mueller),NaN,NaN,1.0,1905 Aldrich St.,Austin,TX,,None,NaN
4,11141496,,Hear Me Bitch 90’s Birthday Bash Comedy Show,Comedy,2019-10-03T19:30:00-05:00,False,None,126,$10 adv / $12 Door,True,...,Fallout Theater,30.269536,-97.745230,1.0,616 Lavaca St,Austin,TX,78701,None,NaN


In [43]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41865 entries, 0 to 41864
Data columns (total 32 columns):
id                  41865 non-null int64
presented_by        36110 non-null object
title               41865 non-null object
category            41865 non-null object
begin_time          41865 non-null object
past                41865 non-null bool
end_time            17708 non-null object
votes               41865 non-null int64
ticket_info         41865 non-null object
is_eventbrite       41865 non-null bool
amazon_affiliate    41865 non-null bool
eventbrite_id       1872 non-null object
is_free             41865 non-null bool
rating              0 non-null object
doors               41865 non-null bool
popularity          41865 non-null float64
sold_out            41865 non-null bool
favorite            41865 non-null bool
artists             41865 non-null object
rsvp                41392 non-null float16
eager               41865 non-null bool
venue.id            41865 non-

In [24]:
#print(ndf.amazon_affiliate.describe())
#print(ndf.favorite.describe())
#print(ndf.rsvp.describe())
#print(ndf.eager.describe())
#print(ndf['giveaway.action'].unique())
#print(ndf['giveaway.ask21'].describe())

[None '/events/11124752/giveaway' '/events/11070390/giveaway'
 '/events/10942317/giveaway' '/events/11035846/giveaway']


In [44]:
# leave out the columns with too many nulls
ndf.drop('rating', axis=1, inplace=True)
ndf.drop('favorite', axis=1, inplace=True)
ndf.drop('rsvp', axis=1, inplace=True)
ndf.drop('eager', axis=1, inplace=True)
ndf.drop('amazon_affiliate', axis=1, inplace=True)
ndf.drop('giveaway.ask21', axis=1, inplace=True)
ndf.drop('giveaway.action', axis=1, inplace=True)
#enforce some cleanup, dtype putting
ndf.past.astype(bool)
ndf.end_time.fillna(value=pd.np.nan, inplace=True)
ndf.is_free.astype(bool)

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


0        False
1        False
2        False
3         True
4        False
5         True
6        False
7        False
8        False
9        False
10       False
11        True
12        True
13        True
14        True
15       False
16        True
17       False
18        True
19       False
20       False
21        True
22        True
23       False
24       False
25       False
26       False
27        True
28       False
29        True
         ...  
41835    False
41836    False
41837    False
41838    False
41839    False
41840    False
41841    False
41842    False
41843    False
41844     True
41845     True
41846    False
41847    False
41848    False
41849    False
41850    False
41851    False
41852     True
41853    False
41854    False
41855    False
41856     True
41857    False
41858    False
41859    False
41860    False
41861    False
41862    False
41863     True
41864    False
Name: is_free, Length: 41865, dtype: bool

In [6]:
pf=ParquetFile('28508revents_processed_part.parq')
ndf=pf.to_pandas()

In [45]:
ndf=df[newcols]
ndf.columns

Index(['id', 'presented_by', 'title', 'category', 'begin_time', 'past',
       'end_time', 'votes', 'ticket_info', 'is_eventbrite', 'amazon_affiliate',
       'eventbrite_id', 'is_free', 'rating', 'doors', 'popularity', 'sold_out',
       'favorite', 'artists', 'rsvp', 'eager', 'venue.id', 'venue.title',
       'venue.latitude', 'venue.longitude', 'venue.popularity',
       'venue.address', 'venue.city', 'venue.state', 'venue.zip',
       'giveaway.action', 'giveaway.ask21'],
      dtype='object')

In [46]:
# insert new columns
ndf['ticket_allages']=[False for x in range(len(ndf))]
ndf['ticket_price_low']=[0 for x in range(len(ndf))]
ndf['ticket_price_max']=[0 for x in range(len(ndf))]
ndf['min_age']=[None for x in range(len(ndf))]
        

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [47]:
# parse ticket info string

tkt_str=ndf.ticket_info.values

In [48]:
from tqdm import tqdm
# dump all info of ticket into other columns
for i,item in tqdm(enumerate(tkt_str)):
    if i%1000==0:print(i)
    #check 'free'
    if "free" in item.lower() : ndf.loc[i,'is_free']=True
    # check child ok        
    if ("all ages" in item.lower()) or ("kids" in item.lower()) or ("child" in item.lower()): ndf.loc[i,'ticket_allages']=True
    
    #replace all whitespace after '$'    
    item = re.sub('\$ ','$',item)
    prices=re.findall('(?<=\$)\d*\.?\d*', item)
    # get all lower/upper $number
    if prices: # not [] or None
        if prices[0].isdigit():
            if len(prices)==1: 
                ndf.loc[i,'ticket_price_low']=prices[0]
                ndf.loc[i,'ticket_price_max']=None
            else:
                #convert all prices to numbers , in case any of them not number
                nprices=[float(x) for x in prices if x.isdigit()]
                
                ndf.loc[i,'ticket_price_low']=min(nprices)
                ndf.loc[i,'ticket_price_max']=max(nprices)      
    # get minimum age
    min_ag=re.findall(r"(\d+)\+",item)
    if min_ag:
        ndf.loc[i,'min_age']=int(min_ag[0])
    else:ndf.loc[i,'min_age']=None

0it [00:00, ?it/s]

0


C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
999it [01:59,  8.72it/s]

1000


1999it [04:00,  9.72it/s]

...

39999it [1:18:11,  7.74it/s]

40000


41000it [1:20:24,  8.30it/s]

41000


41865it [1:22:20,  8.47it/s]


In [14]:
ndf.columns

Index(['id', 'presented_by', 'title', 'category', 'begin_time', 'past',
       'end_time', 'votes', 'ticket_info', 'is_eventbrite', 'amazon_affiliate',
       'eventbrite_id', 'is_free', 'rating', 'doors', 'popularity', 'sold_out',
       'favorite', 'artists', 'rsvp', 'eager', 'venue.id', 'venue.title',
       'venue.latitude', 'venue.longitude', 'venue.popularity',
       'venue.address', 'venue.city', 'venue.state', 'venue.zip',
       'giveaway.action', 'giveaway.ask21', 'ticket_allages',
       'ticket_price_low', 'ticket_price_max', 'min_age'],
      dtype='object')

In [50]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41865 entries, 0 to 41864
Data columns (total 36 columns):
id                  41865 non-null int64
presented_by        36110 non-null object
title               41865 non-null object
category            41865 non-null object
begin_time          41865 non-null object
past                41865 non-null bool
end_time            17708 non-null object
votes               41865 non-null int64
ticket_info         41865 non-null object
is_eventbrite       41865 non-null bool
amazon_affiliate    41865 non-null bool
eventbrite_id       1872 non-null object
is_free             41865 non-null bool
rating              0 non-null object
doors               41865 non-null bool
popularity          41865 non-null float64
sold_out            41865 non-null bool
favorite            41865 non-null bool
artists             41865 non-null object
rsvp                41392 non-null float16
eager               41865 non-null bool
venue.id            41865 non-

In [52]:
# remove ticket info
ndf.drop('giveaway.action', axis=1, inplace=True)
ndf.drop('giveaway.ask21', axis=1, inplace=True)
ndf.drop('ticket_info', axis=1, inplace=True)

In [53]:
ndf.to_csv('fullevents_proc_tickets_art.csv',index=False)

In [49]:
write('fullevent_proc_free_tickets.parq',ndf)

ValueError: Can't infer object conversion type: 0     0
1     0
2     0
3     0
4    10
5     0
6     0
7    10
8    16
9     8
Name: ticket_price_low, dtype: object

In [24]:
# get artist column andef
e_artist=ndf.artists.values

In [25]:
len(e_artist)

28508

In [26]:
import json
json.loads(dumps(e_artist[1177][0]))#['popularity']

{'id': 55257,
 'title': 'Ramsay Midwood',
 'permalink': '/artists/ramsay-midwood',
 'data_path': '/artists/55257/quickie',
 'description': 'From Ramsay Midwood\'s website: "Ramsay Midwood is a miracle and a mystery. There is almost no usable information about him."\r \r It is known that he sings, he plays guitar, he resides in Austin, he has, on occasion, acted and/or played with The Gourds, and he\'s big in Germany. <a href="http://www.last.fm/music/Ramsay+Midwood" rel="nofollow">Listen at Last.fm</a>',
 'hometown': 'Austin, TX',
 'popularity': 31.0,
 'youtube_id': 'uad9NrjQAxw',
 'spotify_id': '5JnyvYVewayggJNe8QRfGQ'}

In [27]:
#get artist_popularity
p_art_avg=[]
p_art_sum=[]
p_art_max=[]
for item in e_artist:
    if len(item)==0:
        p_art_sum+=[0]
        p_art_avg+=[0]
        p_art_max+=[0]
    elif len(item)==1:
        p_art_sum+=[json.loads(dumps(item))[0]['popularity']]
        p_art_avg+=[json.loads(dumps(item))[0]['popularity']]
        p_art_max+=[json.loads(dumps(item))[0]['popularity']]
    else:
        subsum=0
        partmax=0
        for sub in item:
            part=json.loads(dumps(item))[0]['popularity']
            subsum+=part
            if part> partmax: partmax=part
        p_art_sum+=[subsum]
        p_art_avg+=[subsum/len(item)]
        p_art_max+=[partmax]

In [28]:
# place the artist popularity back to df
ndf['artist.popularity.sum']=p_art_sum
ndf['artist.popularity.avg']=p_art_avg
ndf['artist.popularity.max']=p_art_max

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [33]:
ndf.tail()

,id,presented_by,title,category,begin_time,past,end_time,votes,is_eventbrite,amazon_affiliate,...,venue.zip,giveaway.action,giveaway.ask21,ticket_allages,ticket_price_low,ticket_price_max,min_age,artist.popularity.sum,artist.popularity.avg,artist.popularity.max
28503,10193099,ONE-2-ONE BAR PRESENTS,The Texas Kgb (Kelly Green Band),Music,2019-02-18T20:00:00-06:00,True,None,3,False,True,...,78704,None,NaN,False,10,NaN,21,10.0,10.0,10.0
28504,10355891,,Anna Larson,Music,2019-02-18T21:30:00-06:00,True,None,1,False,True,...,78701,None,NaN,False,0,0.0,None,7.0,7.0,7.0
28505,10415703,,"Secret Levels, PoolBoi Blu, Anthony Miguel, an...",Music,2019-02-18T21:50:00-06:00,True,2019-02-19T01:00:00-06:00,2,False,True,...,78701,None,NaN,False,0,0.0,None,9.0,3.0,3.0
28506,10411421,,President's Day Party,Music,2019-02-18T16:00:00-06:00,True,None,5,False,True,...,,None,NaN,False,0,0.0,None,6.0,3.0,3.0
28507,10385427,,The Bulb Story Slam,Comedy,2019-02-18T19:00:00-06:00,True,None,7,False,True,...,78702,None,NaN,False,5,20.0,None,0.0,0.0,0.0


In [31]:
ndf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28508 entries, 0 to 28507
Data columns (total 38 columns):
id                       28508 non-null int64
presented_by             23999 non-null object
title                    28508 non-null object
category                 28508 non-null object
begin_time               28508 non-null object
past                     28508 non-null bool
end_time                 12697 non-null object
votes                    28508 non-null int64
is_eventbrite            28508 non-null bool
amazon_affiliate         28508 non-null bool
eventbrite_id            1294 non-null object
is_free                  28508 non-null bool
rating                   0 non-null object
doors                    28508 non-null bool
popularity               28508 non-null float64
sold_out                 28508 non-null bool
favorite                 28508 non-null bool
artists                  28508 non-null object
rsvp                     28146 non-null float32
eager            

In [38]:
ndf.to_csv('28508_proc_ticket_free.csv')

In [32]:
# save out to tables
from fastparquet import write
write('fullevents_processed_part.parq',ndf)

ValueError: Can't infer object conversion type: 0     0
1     0
2     0
3     0
4    10
5     0
6     0
7    10
8    16
9     8
Name: ticket_price_low, dtype: object

In [34]:
ndf.head()

,id,presented_by,title,category,begin_time,past,end_time,votes,ticket_info,is_eventbrite,...,venue.longitude,venue.popularity,venue.address,venue.city,venue.state,venue.zip,ticket_allages,min_age,ticket_price_low,ticket_price_max
0,11049447,Official 2019 ACL Fest Late Night Show:,Gary Clark Jr,Music,2019-10-03T19:00:00-05:00,False,None,784,All Ages,False,...,-97.736175,15.0,801 Red River,Austin,TX,78701,True,NaN,NaN,NaN
1,11049456,Official 2019 ACL Fest Late Night Show:,Hippie Sabotage w/ Sebastian Paul,Music,2019-10-03T20:00:00-05:00,False,None,223,18+,False,...,-97.728516,15.0,2015 E. Riverside Dr.,Austin,TX,78741,NaN,18.0,NaN,NaN
2,11049430,Official 2019 ACL Fest Late Night Show:,FIDLAR w/ XETAS,Music,2019-10-03T18:30:00-05:00,False,None,163,All Ages,False,...,-97.729385,15.0,1308 E. Fourth,Austin,TX,78702,True,NaN,NaN,NaN
3,11186444,Misfits Comedy Presents,That Joke's New! Comedy Open-Mic,Comedy,2019-10-03T20:00:00-05:00,False,None,151,Free,False,...,NaN,1.0,1905 Aldrich St.,Austin,TX,,NaN,NaN,NaN,NaN
4,11141496,,Hear Me Bitch 90’s Birthday Bash Comedy Show,Comedy,2019-10-03T19:30:00-05:00,False,None,126,$10 adv / $12 Door,True,...,-97.745230,1.0,616 Lavaca St,Austin,TX,78701,NaN,NaN,10.0,12.0


In [227]:
ndf.ticket_info.unique()

array(['All Ages', 'All Ages ', '', '$5', '$22-$25', '$6 / All Ages',
       '$10 / 21+ ', '21+ ', 'Donate ', '21+',
       'Free / All-Ages / Fueled by Lagunitas!', 'Free',
       '$10.00 - $15.00', '18+', '21+ No Cover', '$12-$15 / 21+ ', '$250',
       '$11.25', 'Free, Open to the Public', '$ 9.00',
       "$25 / person (Free for all players participating in COVER 3's 2019 Invitational Golf Classic)",
       '$35 with code YAYAMY', '$10',
       'Tasting Glass w/ Four Pours $9-$20', '$20.00 - $23.00', '$150',
       'Free and All Ages', '$8/ADV $10/DOOR', '21+ / $5 Cover', '$265 ',
       '$25', '$125', '$12, 18+', 'No Cover, 21+', '$35', '$30', '$20',
       '$125 21+', '$58', '6 weeks for $150, Adults', '$50', '$3',
       'ALL AGES', '$3 / 21+', '$30-$50', '$14.99 / All Ages ',
       'No Cover | 21+', '21+ Free / $5 Cover for Minors ', 'Free! RSVP',
       '$5 online / $7 door | BYOB ', 'All Ages / $7-$10 ',
       'Free | All Ages', 'Free / 21+', '$8 ', '$5-$8', '$15', '$ 11.25

In [293]:
ndf.drop('ticket_info', axis=1, inplace=True)

In [255]:
ndf['venue.latitude'].fillna(0,inplace=True)
ndf['venue.longitude'].fillna(0,inplace=True)

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [257]:
ndf['venue.address'].fillna("unknown",inplace=True)
ndf['venue.zip'].fillna(0,inplace=True)

In [289]:
aa=[ x[0:5] if x!=0 else x for x in ndf['venue.zip'].values]
p=[ int(x) if x!='' else 0 for x in aa]

In [291]:
ndf['venue.zip']=p

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [320]:
bands=ndf.presented_by.values

In [321]:
bands

array(['Gay.Do512 & C3 Present', 'C3 Presents', "Barrel O'Fun Presents ",
       ..., 'Mike Webley Presents', '', ''], dtype=object)

In [374]:
# parse the presented_by column and then use the mininum string
nbands=[]
for item in bands:
    k=item
    if (item!='') and (type(item)!=type(None)):nbands+=[item.lower().split('present')[0]]
    else: nbands+=[None]

In [375]:
ndf.presented_by=nbands

In [302]:
# parse artist name
art_p=ndf.artists.values

In [339]:
nartist=[]
lstart=[]
for item in art_p:
    nartist+=[len(item)]
    for sub in item:
        s=json.loads(dumps(sub))
        lstart+=[s['title']]

In [136]:
ndf.category.value_counts()

Music                          470
Film + TV                      102
Community                      100
Food + Drink                    72
Comedy                          71
Art & Culture                   70
Sports + Activities             45
DJ's + Parties                  44
Literary                        34
Workshops + Classes             29
Karaoke + Trivia                17
Variety / Other                 15
Theater                         12
Happy Hour + Drink Specials      7
LGBTQ+                           7
Activism                         3
Exhibit                          1
Name: category, dtype: int64

In [ ]:
# plot event types bar chart


In [127]:
ndf.popularity=ndf.popularity.astype(int)

C:\Users\z0043H7B\AppData\Local\Continuum\miniconda3\lib\site-packages\pandas\core\generic.py:5096: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [377]:
# temporary dump
from fastparquet import write
write('past1yrevents_processed.parq',ndf)

In [1]:
import pandas as pd

In [2]:
from fastparquet import ParquetFile
pf= ParquetFile('past3monthevents_processed.parq')
nndf = pf.to_pandas()

In [4]:
nndf.columns

Index(['id', 'presented_by', 'title', 'category', 'begin_time', 'past',
       'end_time', 'votes', 'is_eventbrite', 'is_free', 'doors', 'popularity',
       'sold_out', 'artists', 'venue.id', 'venue.title', 'venue.longitude',
       'venue.popularity', 'venue.address', 'venue.city', 'venue.state',
       'venue.zip', 'ticket_allages', 'ticket_price_low', 'ticket_price_max',
       'min_age', 'free', 'venue.latitude'],
      dtype='object')

In [5]:
nndf.ticket_price

,id,presented_by,title,category,begin_time,past,end_time,votes,is_eventbrite,is_free,...,venue.address,venue.city,venue.state,venue.zip,ticket_allages,ticket_price_low,ticket_price_max,min_age,free,venue.latitude
0,10840264,gay.do512 & c3,SOFI TUKKER - R.I.P. Shame World Tour w/ Haiku...,Music,2019-10-16T19:00:00-05:00,False,None,414,False,False,...,801 Red River,Austin,TX,78701,True,0.0,0.0,NaN,False,30.268458
1,10732673,c3,The Distillers w/ Death Valley Girls,Music,2019-10-16T20:00:00-05:00,False,None,157,False,False,...,2015 E. Riverside Dr.,Austin,TX,78741,True,0.0,0.0,NaN,False,30.240266
2,11211745,barrel o'fun,Over the Rainbow,Comedy,2019-10-16T20:00:00-05:00,False,None,67,False,True,...,1911 Aldrich St. Suite 120,Austin,TX,0,False,0.0,0.0,NaN,False,0.000000
3,11219464,None,The Chip Show Live - Comedy,Comedy,2019-10-16T20:00:00-05:00,False,None,59,False,False,...,2908 Fruth St,Austin,TX,78705,False,5.0,NaN,NaN,False,30.295495
4,11218114,None,Carrie Rodriguez,Music,2019-10-16T20:00:00-05:00,False,None,14,False,False,...,2247 Guadalupe St.,Austin,TX,78705,False,22.0,25.0,NaN,False,30.286514


In [6]:
nndf.to_csv('testout_3month_processed.csv')